In [1]:
import pandas as pd
import numpy as np

from itertools import tee, izip
import os

import ConfigParser

from multiprocessing import Manager

# Environment variables store in config file - depends on where we run this ntbk

In [2]:
config = ConfigParser.ConfigParser()
#config.read("/user_home/w_howardx/git/FacebookV/config/wakari.conf") # wakari
config.read("/home/ec2-user/Kaggle/facebook_Jul_2016/FacebookV/config/aws.conf") # AWS

param_obj = ConfigParser.ConfigParser()
param_obj.read(config.get("path", "param_file"))

['/home/ec2-user/Kaggle/facebook_Jul_2016/FacebookV/params/param_1.conf']

In [3]:
print config.get("path", "test_grid_path")
print config.sections()
print config.options("path")

/home/ec2-user/Kaggle/facebook_Jul_2016/feature_engineered_input/hr_wk_mth/50_50_grid/test/
['path']
['train_file', 'test_file', 'param_file', 'train_grid_path', 'test_grid_path', 'model_bite_path', 'model_txt_path', 'grid_output_path', 'output_path']


In [4]:
param = dict(param_obj.items('param'))
print param

{'silent': '1', 'eval_metric': 'mlogloss', 'nthread': '4', 'eta': '0.1', 'objective': 'multi:softprob', 'booster': 'gbtree'}


In [5]:
train = pd.read_csv(config.get("path", "train_file"))
test = pd.read_csv(config.get("path", "test_file"))

# for smaller wakari instance
#train = train.head(5000)
#test = test.head(5000)
print train.shape

# for real testing set we don't need to do this - it doesn't have place_id info
#test = test[['row_id', 'x', 'y', 'accuracy', 'time']]

print train.head()
print test.head()

(29118021, 9)
   row_id       x       y  accuracy    time    place_id  hour  weekday  month
0       0  0.7941  9.0809        54  470702  8523065625    22        5     11
1       1  5.9567  4.7968        13  186555  1757726713    14        4      5
2       2  8.3078  7.0407        74  322648  1137537235     2        1      8
3       3  7.3665  2.5165        65  704587  6567393236     8        7      5
4       4  4.0961  1.1307        31  472130  7440663949    21        6     11
   row_id       x       y  accuracy    time  hour  weekday  month
0       0  0.1675  1.3608       107  930883    11        3     10
1       1  7.3909  2.5301        35  893017     4        5      9
2       2  8.0978  2.3473        62  976933    11        7     11
3       3  0.9990  1.0591        62  907285     2        1     10
4       4  0.6670  9.7254        40  914399    24        5     10


In [6]:
#import K_fbV.fb_split_grid as sg # single process split grid - with normal dictionary as output
import K_fbV.para_split_grid as psg # multi-process split grid - with multiprocess.Manager.dict() object as output
import K_fbV.factorize_predictor as fp

In [7]:
# generate the grids
train_grid, test_grid = psg.get_grids(train, test, outputFile = True,
                                    train_output = config.get("path", "train_grid_path"),
                                    test_output = config.get("path", "test_grid_path"), n = 50, m = 50)

print train_grid[(2,3)].shape # number of testing set data points in grid (2, 3)
print type(train_grid)

(10794, 9)
<class 'multiprocessing.managers.DictProxy'>


In [8]:
import K_fbV.feature_predictor_split as fps
import K_fbV.tune_numRound_maxDepth as tnm

import K_fbV.train_grid_model as tgm
import K_fbV.make_grid_prediction as mgp

import K_fbV.gen_submission as gs

# take only top 80 percentile in all training set grids for modeling - WRONG

In [9]:
import K_fbV.top_k_percentil_in_grid as tkpig

for tup in train_grid.keys():
    train_grid[tup] = tkpig.top_K_percentile_pid(train_grid[tup], k_p = 80, pid = 'place_id')
    
print train_grid[(2,3)].shape # number of training set data points in grid (2, 3)

(8654, 9)


In [11]:
#from multiprocessing import Pool
#import itertools

#p = Pool()                                   #number of processes = number of CPUs
#keys, values= zip(*train_grid.iteritems())            #ordered keys and values
#processed_values = p.map( tkpig.top_K_percentile_pid, 
#                         itertools.izip
#                        (
#                            train_grid.keys(),
#                            itertools.repeat(80),
#                            itertools.repeat('place_id') 
#                        ))

In [10]:
param['eta'] = float(param['eta']) # Parameters for Tree Booster - Booster parameter
param['silent'] = int(param['silent']) # whether to print logs
param['nthread'] = int(param['nthread']) # parallelism

#Quick smoke test of the entire pipeline

In [13]:
manager = Manager()

model_dict = manager.dict() # dictionary object can be shared by multiple processes
unique_class_dict = manager.dict() # dictionary object can be shared by multiple processes

features = ['x', 'y', 'accuracy', 'time']
tgm.train_model((2, 3), train_grid, param, model_dict, unique_class_dict,
            feature_list = features, predictor_name = 'place_id',
            dump_model_txt = True, model_txt_file = config.get("path", "model_txt_path"),
            save_model = True, save_model_file = config.get("path", "model_bite_path"),
            leaf_mltpr = 3.01, tree_mltpr = 8.01)

features = ['x', 'y', 'accuracy', 'time']
tgm.train_model((2, 2), train_grid, param, model_dict, unique_class_dict,
            feature_list = features, predictor_name = 'place_id',
            dump_model_txt = True, model_txt_file = config.get("path", "model_txt_path"),
            save_model = True, save_model_file = config.get("path", "model_bite_path"),
            leaf_mltpr = 1.01, tree_mltpr = 1.01)

[0]	train-mlogloss:2.940825
[1]	train-mlogloss:2.636693


number of boosters in training: 3
number of boosters in training: 1

[2]	train-mlogloss:2.422359
[0]	train-mlogloss:3.212625


In [14]:
prediction_dict = manager.dict() # dictionary object can be shared by multiple processes

#features = ['x', 'y', 'accuracy', 'time']
features = ['x', 'y', 'accuracy', 'time', 'hour', 'weekday', 'month']
mgp.predict_output((2, 3), test_grid, model_dict, unique_class_dict, prediction_dict,
               test_row_id = 'row_id', feature_list = features,
               saveFile = True, outputPath = config.get("path", "grid_output_path"))

#features = ['x', 'y', 'accuracy', 'time']
features = ['x', 'y', 'accuracy', 'time', 'hour', 'weekday', 'month']
mgp.predict_output((2, 2), test_grid, model_dict, unique_class_dict, prediction_dict,
               test_row_id = 'row_id', feature_list = features,
               saveFile = True, outputPath = config.get("path", "grid_output_path"))

In [15]:
finalResult = gs.merge_prediction_grids(prediction_dict, save_file = True, output_path = config.get("path", "output_path"))
print finalResult.head()

      row_id             0             1             2
2836      95  9.682966e+09  3.513194e+09  7.809444e+09
0       1698  2.405501e+09  8.617309e+09  7.964007e+09
1       1726  4.608206e+09  8.050670e+09  1.657333e+09
2837    2354  3.327668e+09  7.268641e+09  1.091456e+09
2       3310  4.682130e+09  3.676443e+09  8.947502e+09


K_fbV/gen_submission.py:8: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  df_finalResult = df_finalResult.sort(['row_id'])


# multi-process Parallalization of the above pipeline to use all cores
- specify a pool of processes using multiprocess package
- need wrapper a function for each function with input parameters more than 1
- use itertools to zip arguments in a list, and in wrapper function unzip/unwrap it

In [ ]:
import time
from functools import wraps

def fn_timer(function):
    @wraps(function)
    def function_timer(*args, **kwargs):
        t0 = time.time()
        result = function(*args, **kwargs)
        t1 = time.time()
        print ("Total time running %s: %s seconds" %
               (function.func_name, str(t1-t0))
               )
        return result
    return function_timer

In [ ]:
def parallel_train_wrap(inputs): # original function wrapper for catching exceptions and unpack input list
    try:
        """Convert f([1,2]) to f(1,2) call"""
        print "traninig a grid..."
        tgm.train_model(*inputs) # original function
    except:
        print inputs[0] # print the tuple value

In [ ]:
import multiprocessing
from multiprocessing import *
from multiprocessing import Queue, Process, freeze_support
from multiprocessing import Pool

import itertools # for multiple input arguments of the function to be parallilized

print multiprocessing.cpu_count() # print the number of CPUs availalbe to system

# define input arguments with their associated names in function
feature_list = ['x', 'y', 'accuracy', 'time']
predictor_name = 'place_id'
leaf_mltpr = 1.002
tree_mltpr = 1.001

In [ ]:
@fn_timer
def timed_parallel_train_wrap():
    pool = Pool( processes = int(multiprocessing.cpu_count()) ) # process pool based on number of cores
    pool.map
    (
        parallel_train_wrap, # function to be parallilized
        itertools.izip
        (
            train_grid.keys(),
            itertools.repeat(train_grid),
            itertools.repeat(param),
            itertools.repeat(model_dict),
            itertools.repeat(unique_class_dict),
            itertools.repeat(feature_list),
            itertools.repeat(predictor_name),
            itertools.repeat(leaf_mltpr),
            itertools.repeat(tree_mltpr)
        )
    )

In [ ]:
print type(model_dict)

In [ ]:
timed_parallel_train_wrap()
print model_dict